In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import (
    Input, Conv2D, SeparableConv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D,
    BatchNormalization, Activation, Dense, Dropout, Flatten, Multiply, Add, Lambda, SpatialDropout2D, Reshape, GlobalMaxPooling2D
)
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import HeNormal, GlorotNormal
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import pathlib
from tensorflow.keras.optimizers.schedules import (
    ExponentialDecay,
    PolynomialDecay,
    PiecewiseConstantDecay,
    CosineDecay,
    CosineDecayRestarts,
    InverseTimeDecay,
    LearningRateSchedule
)

In [ ]:
!pip install gdown
import gdown

file_id = ''
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'downloaded_file.zip', quiet=False)

In [3]:
!unzip -q /kaggle/working/downloaded_file.zip -d extracted_folder

In [4]:
def create_dataset(data_dir, batch_size=16, validation_split=0.2):
    """Create train and validation datasets with a split."""
    data_dir = pathlib.Path(data_dir)
    all_image_paths = list(data_dir.glob('*/*.jpg'))
    all_image_paths = [str(path) for path in all_image_paths]
    class_names = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])
    class_names = tf.constant(class_names)

    # Shuffle the paths
    total_images = len(all_image_paths)
    tf.random.set_seed(42)  # Ensure reproducibility
    all_image_paths = tf.random.shuffle(all_image_paths)

    # Compute split index
    val_size = int(total_images * validation_split)
    train_paths = all_image_paths[val_size:]
    val_paths = all_image_paths[:val_size]

    # Create label functions
    def get_label(file_path):
        parts = tf.strings.split(file_path, '/')
        class_name = parts[-2]
        label = tf.argmax(tf.cast(class_names == class_name, tf.int32))
        return label

    def process_image(file_path, label, augment=False):
        image = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])

        if augment:
            # Apply data augmentation
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_brightness(image, max_delta=0.2)
            image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
            image = tf.image.random_saturation(image, lower=0.8, upper=1.2)

        image = image / 127.5 - 1.0
        return image, label

    def prepare_dataset(paths, augment=False):
        paths_ds = tf.data.Dataset.from_tensor_slices(paths)
        labels_ds = paths_ds.map(get_label)
        dataset = tf.data.Dataset.zip((paths_ds, labels_ds))
        dataset = dataset.map(lambda x, y: process_image(x, y, augment=augment))
        dataset = dataset.shuffle(buffer_size=1000)
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        return dataset

    # Create train and validation datasets
    train_dataset = prepare_dataset(train_paths, augment=True)  # Augmentation applied
    val_dataset = prepare_dataset(val_paths, augment=False)     # No augmentation for validation

    return train_dataset, val_dataset, class_names

In [5]:
# Define parameters
data_dir = "extracted_folder/content/food_dataset/data"
batch_size = 128
train_dataset, val_dataset, class_names = create_dataset(data_dir, batch_size=batch_size)

In [ ]:
for images, labels in train_dataset.take(1):  # Fetch one batch
    print(f"Images shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"First label in the batch: {labels[0]}")

# Display some images with labels
def display_images_with_labels(dataset, class_names, num_images=15):
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):
        for i in range(num_images):
            ax = plt.subplot(3, 5, i + 1)
            plt.imshow(images[i].numpy())
            plt.title(class_names[labels[i].numpy()].numpy().decode("utf-8"))
            plt.axis("off")
    plt.show()

# Call the function to display images
display_images_with_labels(train_dataset, class_names)

In [7]:
def se_block(input_tensor, reduction=16):
    filters = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // reduction, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(1e-3))(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer=GlorotNormal(), kernel_regularizer=l2(1e-3))(se)
    return Multiply()([input_tensor, se])

In [8]:
def cbam_block(input_tensor, reduction_ratio=16):
    """CBAM block for feature refinement."""
    # Channel Attention
    channel_avg_pool = GlobalAveragePooling2D()(input_tensor)
    channel_max_pool = GlobalMaxPooling2D()(input_tensor)

    channel_avg_pool = Dense(input_tensor.shape[-1] // reduction_ratio, activation='relu')(channel_avg_pool)
    channel_avg_pool = Dense(input_tensor.shape[-1], activation='sigmoid')(channel_avg_pool)

    channel_max_pool = Dense(input_tensor.shape[-1] // reduction_ratio, activation='relu')(channel_max_pool)
    channel_max_pool = Dense(input_tensor.shape[-1], activation='sigmoid')(channel_max_pool)

    channel_avg_pool = Reshape((1, 1, input_tensor.shape[-1]))(channel_avg_pool)
    channel_max_pool = Reshape((1, 1, input_tensor.shape[-1]))(channel_max_pool)

    channel_attention = Add()([channel_avg_pool, channel_max_pool])
    channel_attention = Multiply()([input_tensor, channel_attention])

    # Spatial Attention
    def spatial_attention(inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        return tf.concat([avg_pool, max_pool], axis=-1)

    spatial_attention = Lambda(spatial_attention)(channel_attention)
    spatial_attention = Conv2D(1, (7, 7), activation='sigmoid', padding='same')(spatial_attention)
    spatial_attention = Multiply()([channel_attention, spatial_attention])

    return spatial_attention

In [9]:
def inception_block_with_residual(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj, scale=0.2):
    # Save the input for residual connection
    shortcut = x

    # 1x1 Convolution
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(1e-3))(x)
    conv_1x1 = BatchNormalization()(conv_1x1)
    conv_1x1 = SpatialDropout2D(0.2)(conv_1x1)

    # 3x3 Convolution
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(1e-3))(x)
    conv_3x3 = BatchNormalization()(conv_3x3)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(1e-3))(conv_3x3)
    conv_3x3 = BatchNormalization()(conv_3x3)
    conv_3x3 = SpatialDropout2D(0.2)(conv_3x3)

    # 5x5 Convolution
    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(1e-3))(x)
    conv_5x5 = BatchNormalization()(conv_5x5)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(1e-3))(conv_5x5)
    conv_5x5 = BatchNormalization()(conv_5x5)
    conv_5x5 = SpatialDropout2D(0.2)(conv_5x5)

    # Max Pooling + 1x1 Convolution
    pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(1e-3))(pool_proj)
    pool_proj = BatchNormalization()(pool_proj)
    pool_proj = SpatialDropout2D(0.2)(pool_proj)

    # Concatenate all branches
    output = tf.keras.layers.Concatenate()([conv_1x1, conv_3x3, conv_5x5, pool_proj])

    # Apply Dropout after concatenation
    output = Dropout(0.3)(output)

    # Add SE block
    output = cbam_block(output)

    # Add residual connection if dimensions match
    if shortcut.shape[-1] != output.shape[-1]:  # Match dimensions using 1x1 conv
        shortcut = Conv2D(output.shape[-1], (1, 1), padding='same', kernel_initializer=HeNormal())(shortcut)
        shortcut = BatchNormalization()(shortcut)

    scaled_output = tf.keras.layers.Lambda(lambda s: s * scale)(output)

    output = tf.keras.layers.Add()([shortcut, scaled_output])
    output = Activation('relu')(output)

    return output

In [10]:
def auxiliary_classifier(x, num_classes):
    aux = AveragePooling2D((5, 5), strides=(3, 3))(x)
    aux = Conv2D(64, (1, 1), padding='same', activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(1e-3))(aux)
    aux = Dropout(0.5)(aux)
    aux = Flatten()(aux)
    aux = Dense(num_classes, activation='softmax', kernel_initializer=HeNormal(), kernel_regularizer=l2(1e-3))(aux)
    return aux

In [11]:
def GoogleNet(input_shape=(224, 224, 3), num_classes=1000):
    input_tensor = Input(shape=input_shape)
    initializer = HeNormal()

    # Initial layers
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', kernel_initializer=initializer, kernel_regularizer=l2(1e-3))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout2D(0.1)(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = Conv2D(64, (1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=l2(1e-3))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout2D(0.1)(x)
    x = Conv2D(192, (3, 3), padding='same', kernel_initializer=initializer, kernel_regularizer=l2(1e-3))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout2D(0.1)(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Inception blocks with auxiliary classifiers
    x = inception_block_with_residual(x, 64, 96, 128, 16, 32, 32)
    aux_output1 = auxiliary_classifier(x, num_classes)

    x = inception_block_with_residual(x, 128, 128, 192, 32, 96, 64)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_block_with_residual(x, 192, 96, 208, 16, 48, 64)
    x = inception_block_with_residual(x, 160, 112, 224, 24, 64, 64)
    x = inception_block_with_residual(x, 128, 128, 256, 24, 64, 64)
    aux_output2 = auxiliary_classifier(x, num_classes)

    x = inception_block_with_residual(x, 112, 144, 288, 32, 64, 64)
    x = inception_block_with_residual(x, 256, 160, 320, 32, 128, 128)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_block_with_residual(x, 256, 160, 320, 32, 128, 128)
    x = inception_block_with_residual(x, 384, 192, 384, 48, 128, 128)
    x = AveragePooling2D((7, 7), strides=(1, 1), padding='valid')(x)

    # Fully connected layers
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer=initializer, kernel_regularizer=l2(1e-3))(x)

    # Final model
    model = tf.keras.Model(inputs=input_tensor, outputs=[x, aux_output1, aux_output2])
    return model

# Create and summarize the model
model = GoogleNet(input_shape=(224, 224, 3), num_classes=101)
#model.summary()

In [12]:
from keras.saving import register_keras_serializable

@register_keras_serializable()
def spatial_attention(inputs):
    # Your spatial attention logic here
    avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
    return tf.concat([avg_pool, max_pool], axis=-1)

In [13]:
from tensorflow.keras.models import load_model
from keras.config import enable_unsafe_deserialization

# Enable unsafe deserialization
enable_unsafe_deserialization()

# Load the model
model = load_model('/kaggle/input/model.keras')

In [14]:
import tensorflow as tf

def sparse_categorical_crossentropy_with_label_smoothing(y_true, y_pred, num_classes, smoothing=0.05):
    """
    Custom sparse categorical crossentropy with label smoothing.

    Args:
        y_true: Tensor of shape (batch_size,) containing true class indices.
        y_pred: Tensor of shape (batch_size, num_classes) containing predicted probabilities.
        num_classes: Number of classes.
        smoothing: Smoothing factor (float between 0 and 1).

    Returns:
        Scalar loss value.
    """
    # Apply label smoothing
    confidence = 1.0 - smoothing
    low_confidence = smoothing / (num_classes - 1)

    # Convert sparse labels to one-hot
    y_true_one_hot = tf.one_hot(y_true, depth=num_classes)

    # Smooth the labels
    # Smooth the labels correctly
    y_true_smoothed = y_true_one_hot * confidence + (1 - y_true_one_hot) * low_confidence

    # Compute cross-entropy loss
    loss = -tf.reduce_sum(y_true_smoothed * tf.math.log(y_pred + 1e-7), axis=-1)
    return tf.reduce_mean(loss)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.000001,  # Initial LR
    decay_steps=250,             # Number of steps after which LR decays
    decay_rate=0.95,              # Decay rate (reduce LR by 4% every 1000 steps)
    staircase=True                # Use discrete (staircase) decay
)
# Updated loss function: sum of main loss and auxiliary losses
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
best_val_acc = 0.8002

# Path to save the best weights
best_weights_filepath = "/kaggle/working/best_weights.weights.h5"
# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Metrics
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
train_loss_metric = tf.keras.metrics.Mean(name="train_loss")
val_loss_metric = tf.keras.metrics.Mean(name="val_loss")

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # Forward pass: Get main and auxiliary predictions
        main_pred, aux_pred1, aux_pred2 = model(images, training=True)

        # Compute individual losses
        main_loss = sparse_categorical_crossentropy_with_label_smoothing(labels, main_pred, 101, smoothing=0)
        aux_loss1 = sparse_categorical_crossentropy_with_label_smoothing(labels, aux_pred1, 101, smoothing=0)
        aux_loss2 = sparse_categorical_crossentropy_with_label_smoothing(labels, aux_pred2, 101, smoothing=0)

        # Combine losses (weight auxiliary losses lower)
        total_loss = main_loss

    # Backward pass: Compute gradients
    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Update metrics
    train_acc_metric.update_state(labels, main_pred)
    train_loss_metric.update_state(total_loss)

    return total_loss

@tf.function
def val_step(images, labels):
    # Forward pass: Get predictions
    main_pred, _, _ = model(images, training=False)

    # Compute loss and update metrics
    val_loss = loss_fn(labels, main_pred)
    val_acc_metric.update_state(labels, main_pred)
    val_loss_metric.update_state(val_loss)

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training loop
    for step, (images, labels) in enumerate(train_dataset):
        loss = train_step(images, labels)

        if step % 100 == 0:
            print(f"Step {step}: Loss = {loss.numpy():.4f}, Accuracy = {train_acc_metric.result().numpy():.4f}")

    # Compute training metrics
    train_acc = train_acc_metric.result()
    train_loss = train_loss_metric.result()
    train_acc_metric.reset_state()
    train_loss_metric.reset_state()

    # Validation loop
    for images, labels in val_dataset:
        val_step(images, labels)

    # Compute validation metrics
    val_acc = val_acc_metric.result()
    val_loss = val_loss_metric.result()
    val_acc_metric.reset_state()
    val_loss_metric.reset_state()

    print(f"Training Accuracy: {train_acc:.4f}, Training Loss: {train_loss:.4f}")
    print(f"Validation Accuracy: {val_acc:.4f}, Validation Loss: {val_loss:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        model.save_weights(best_weights_filepath)  # Save the best weights to a file
        print(f"Saved best weights with validation accuracy: {best_val_acc:.4f}")
model.load_weights(best_weights_filepath)
print(f"Restored best weights from file with validation accuracy: {best_val_acc:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get a batch of images and labels from the validation dataset
for images, labels in val_dataset.take(1):  # Take one batch
    # Get predictions
    main_pred, _, _ = model(images, training=False)

    # Convert predictions to class labels
    predicted_labels = tf.argmax(main_pred, axis=1)

    # Plot images and show predicted vs. true labels
    plt.figure(figsize=(12, 12))
    for i in range(9):  # Display first 9 images
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"True: {labels[i].numpy()}, Pred: {predicted_labels[i].numpy()}")
        plt.axis("off")
    plt.show()

In [ ]:
model.load_weights(best_weights_filepath)
print(f"Restored best weights from file with validation accuracy: {best_val_acc:.4f}")

In [18]:
model.save('model.keras')